In [1]:
BASE_DIR = '../'*3
DATASET_DIR = BASE_DIR+'code/util/dataset/dataset_dummy.ipynb'
EXTMODEL_DIR = BASE_DIR + 'code/CNN/cnn_ext_model/ext_model.ipynb'

In [2]:
%run {DATASET_DIR}
%run {EXTMODEL_DIR}

Using device: cpu
Using PyTorch version: 1.9.0


In [3]:
imagenet = DummyDataset('imagenet', 'select',[3,299,299],200)

200


In [4]:
CnnExtModel.set_macro('v3_preproc', 
          ['serial',
              ['conv',{'ksize':3,'stride':2,'chn':32, 'padding':'VALID'}],
              ['conv',{'ksize':3,'chn':32, 'padding':'VALID'}],
              ['conv',{'ksize':3,'chn':64, 'padding':'SAME'}],
              ['max',{'ksize':3, 'stride':2, 'padding':'VALID'}],
              ['conv',{'ksize':1,'chn':80,'padding':'VALID'}],
              ['conv',{'ksize':3,'chn':192,'padding':'VALID'}],
              ['max',{'ksize':3, 'stride':2,'padding':'VALID'}]
          
          ]
                     )

In [5]:
CnnExtModel.set_macro('v3_inception1',
        ['parallel',
            ['conv',{'ksize':1, 'chn':64}],
         
            ['serial',
                ['conv',{'ksize':1,'chn':48}],
                ['conv',{'ksize':5, 'chn':64}]],
         
            ['serial',
                ['conv',{'ksize':1,'chn':64}],
                 ['conv',{'ksize':3,'chn':96}],
                 ['conv',{'ksize':3,'chn':96}]],
         
            ['serial',
                ['avg',{'ksize':3,'stride':1}],
                ['conv',{'ksize':1,'chn':'#chn'}]]
        
        ]
                    
                     )

In [6]:
CnnExtModel.set_macro('v3_resize1',
            ['parallel',
                ['conv',{'ksize':3,'stride':2,'chn':384}],
                ['serial',
                 ['conv',{'ksize':1,'chn':64}],
                 ['conv',{'ksize':3,'chn':94}],
                 ['conv',{'ksize':3,'stride':2,'chn':96}]],
                ['max',{'ksize':3,'stride':2}]
            
            ]                     
                    
                     )

In [7]:
CnnExtModel.set_macro('v3_inception2',
    ['parallel',
        ['conv', {'ksize':1, 'chn':192}],
        ['serial',
            ['conv', {'ksize':[1,1], 'chn':'#chn'}],
            ['conv', {'ksize':[1,7], 'chn':'#chn'}],
            ['conv', {'ksize':[7,1], 'chn':192}]],
        ['serial',
            ['conv', {'ksize':[1,1], 'chn':'#chn'}],
            ['conv', {'ksize':[7,1], 'chn':'#chn'}],
            ['conv', {'ksize':[1,7], 'chn':'#chn'}],
            ['conv', {'ksize':[7,1], 'chn':'#chn'}],
            ['conv', {'ksize':[1,7], 'chn':192}]],
        ['serial',
            ['avg', {'ksize':3, 'stride':1}],
            ['conv', {'ksize':1, 'chn':192}]]])

In [8]:
CnnExtModel.set_macro('v3_resize2',
    ['parallel',
        ['serial',
            ['conv', {'ksize':1, 'chn':192}],
            ['conv', {'ksize':3, 'stride':2, 'chn':320}]],
        ['serial',
            ['conv', {'ksize':[1,1], 'chn':192}],
            ['conv', {'ksize':[1,7], 'chn':192}],
            ['conv', {'ksize':[7,1], 'chn':192}],
            ['conv', {'ksize':[3,3], 'stride':[2,2], 'chn':192}]],
        ['max', {'ksize':3, 'stride':2}]])

In [9]:
CnnExtModel.set_macro('v3_inception3',
    ['parallel',
        ['conv', {'ksize':1, 'chn':320}],
        ['serial',
            ['conv', {'ksize':[3,3], 'chn':384}],
            ['parallel',
                ['conv', {'ksize':[1,3], 'chn':384}],
                ['conv', {'ksize':[3,1], 'chn':384}]]],
        ['serial',
            ['conv', {'ksize':[1,1], 'chn':448}],
            ['conv', {'ksize':[3,3], 'chn':384}],
            ['parallel',
                ['conv', {'ksize':[1,3], 'chn':384}],
                ['conv', {'ksize':[3,1], 'chn':384}]]],
        ['serial',
            ['avg', {'ksize':3, 'stride':1}],
            ['conv', {'ksize':1, 'chn':192}]]])

In [10]:
CnnExtModel.set_macro('v3_postproc',
    ['serial',
        ['avg', {'stride':8}],
        ['dropout', {'keep_prob':0.7}]])

In [11]:
CnnExtModel.set_macro('inception_v3',
    ['serial',
        ['custom', {'name':'v3_preproc'}],
        ['custom', {'name':'v3_inception1', 'args':{'#chn':32}}],
        ['custom', {'name':'v3_inception1', 'args':{'#chn':64}}],
        ['custom', {'name':'v3_inception1', 'args':{'#chn':64}}],
        ['custom', {'name':'v3_resize1'}],
        ['custom', {'name':'v3_inception2', 'args':{'#chn':128}}],
        ['custom', {'name':'v3_inception2', 'args':{'#chn':160}}],
        ['custom', {'name':'v3_inception2', 'args':{'#chn':160}}],
        ['custom', {'name':'v3_inception2', 'args':{'#chn':192}}],
        ['custom', {'name':'v3_resize2'}],
        ['custom', {'name':'v3_inception3'}],
        ['custom', {'name':'v3_inception3'}],
        ['custom', {'name':'v3_postproc'}]])

In [12]:
inception_v3 = CnnExtModel('inception_v3', imagenet,
                   [['custom', {'name':'inception_v3'}]], dump_structure=True)

custom inception_v3
  serial
    custom v3_preproc
      serial
        1: conv, [3, 299, 299]=>[32, 148, 148]pm : 32x3x3x3+32 = 896
        2: conv, [32, 148, 148]=>[32, 146, 146]pm : 32x32x3x3+32 = 9248
        3: conv, [32, 146, 146]=>[64, 146, 146]pm : 64x32x3x3+64 = 18496
        4: max, [64, 146, 146]=>[64, 72, 72]
        5: conv, [64, 72, 72]=>[80, 72, 72]pm : 80x64x1x1+80 = 5200
        6: conv, [80, 72, 72]=>[192, 70, 70]pm : 192x80x3x3+192 = 138432
        7: max, [192, 70, 70]=>[192, 34, 34]
    custom v3_inception1
      parallel
        8: conv, [192, 34, 34]=>[64, 34, 34]pm : 64x192x1x1+64 = 12352
        serial
          9: conv, [192, 34, 34]=>[48, 34, 34]pm : 48x192x1x1+48 = 9264
          10: conv, [48, 34, 34]=>[64, 34, 34]pm : 64x48x5x5+64 = 76864
        serial
          11: conv, [192, 34, 34]=>[64, 34, 34]pm : 64x192x1x1+64 = 12352
          12: conv, [64, 34, 34]=>[96, 34, 34]pm : 96x64x3x3+96 = 55392
          13: conv, [96, 34, 34]=>[96, 34, 34]pm : 96x96x3x3

          103: conv, [448, 8, 8]=>[384, 8, 8]pm : 384x448x3x3+384 = 1548672
          parallel
            104: conv, [384, 8, 8]=>[384, 8, 8]pm : 384x384x1x3+384 = 442752
            105: conv, [384, 8, 8]=>[384, 8, 8]pm : 384x384x3x1+384 = 442752
        serial
          106: avg, [2048, 8, 8]=>[2048, 8, 8]
          107: conv, [2048, 8, 8]=>[192, 8, 8]pm : 192x2048x1x1+192 = 393408
    custom v3_postproc
      serial
        108: avg, [2048, 8, 8]=>[2048, 1, 1]
        109: dropout, [2048, 1, 1]=>[2048, 1, 1]
110: full, [2048, 1, 1]=>[200]pm :200x2048+200=409800
Total parameter count : 32398886


In [26]:
#'1. 최초 블럭하나가 와서 모듈이 4개면 총 4개로 리스트에 다긴다.
name = 'v3_inception3_11'

class tmp(nn.Module):
    def __init__(self, pm_hidden, block_name, idx):
        
        setattr(self, block_name, nn.Sequential())
                
        block=[]
#         self.add_module()
        for name, module in pm_hidden.named_children():
            if not isinstance(module, nn.Sequential):
                block.append((name, module))

            
            else:
                
                for node, module_node in list(module.named_modules())[1:]:
                    node = node.replace('.','_')
                    setattr(self, node, module_node)
                
        method=getattr(self, block_name)
        method(*block)



    def forward(self,x):
        
    
#'2. 4개의 블럭을 named모듈로 검사하여 palalell이라는 단어가 있으면
#'3. 다시 리스트에 담에서 병렬처리를 하게 한다.

# inception_v3.modules[10].

4

In [175]:
i.i=1

AttributeError: 'str' object has no attribute 'i'

In [167]:
inception_v3.modules[10].parallel_91.serial_89

Conv2d(1280, 384, kernel_size=(3, 3), stride=(1, 1), padding=same)

In [176]:
for name, i in inception_v3.modules[10].named_modules():
    print(name)
    break

In [174]:
inception_v3.modules[10]

Sequential(
  (parallel_88): Conv2d(1280, 320, kernel_size=(1, 1), stride=(1, 1), padding=same)
  (parallel_91): Sequential(
    (serial_89): Conv2d(1280, 384, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (serial_91): Sequential(
      (parallel_90): Conv2d(384, 384, kernel_size=(1, 3), stride=(1, 1), padding=same)
      (parallel_91): Conv2d(384, 384, kernel_size=(3, 1), stride=(1, 1), padding=same)
    )
  )
  (parallel_95): Sequential(
    (serial_92): Conv2d(1280, 448, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (serial_93): Conv2d(448, 384, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (serial_95): Sequential(
      (parallel_94): Conv2d(384, 384, kernel_size=(1, 3), stride=(1, 1), padding=same)
      (parallel_95): Conv2d(384, 384, kernel_size=(3, 1), stride=(1, 1), padding=same)
    )
  )
  (parallel_97): Sequential(
    (serial_96): AvgPool2d(kernel_size=(1, 1), stride=(1, 1), padding=0)
    (serial_97): Conv2d(1280, 192, kernel_size=(1, 1), stride=(1, 1

In [179]:
for name, module in inception_v3.modules[10].named_children():
    if name == 'parallel_91':
        
        
        
        for i,j in list(module.named_modules())[1:]:
            print(i)
#             break

        
#         modul
# #         module[0]
# #         print(list(module.state_dict()))
# #         a=list(map(lambda x : len(x.split('.')), list(module.state_dict())))
#         b=list(map(lambda x : x.split('.')[:-1], list(module.state_dict())))
#         tmp = []
#         for i in b:
#             chck = None
#             if chck == i :
#                 pass
#             else:
#                 getattr(modul)
            
#             print(i)
#         print(max(a))

#         print(list(module.modules()))

serial_89
serial_91
serial_91.parallel_90
serial_91.parallel_91


In [182]:
'serial_89'.replace('.','_')
# 'serial_91.parallel_90'.replace('.','_')

'serial_89'

In [141]:
for name, modules in list(inception_v3.modules[10]):
    print(name)

TypeError: cannot unpack non-iterable Conv2d object

In [399]:
for i in range(len(list(inception_v3.modules[10]))):
#     print(list(inception_v3.modules[10])[i])
    
    if isinstance(list(inception_v3.modules[10])[i],nn.Sequential):
        for j,f in list(inception_v3.modules[10])[i].named_modules():
            print(f)
    
    
    
#     for j in list(inception_v3.modules[10])[i]
    if i == 1:
        break
#     for j in [list(inception_v3.modules[10])[i]]:
#         print(j)

Sequential(
  (serial_0): Conv2d(1280, 384, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (serial_1): Sequential(
    (parallel_0): Conv2d(384, 384, kernel_size=(1, 3), stride=(1, 1), padding=same)
    (parallel_1): Conv2d(384, 384, kernel_size=(3, 1), stride=(1, 1), padding=same)
  )
)
Conv2d(1280, 384, kernel_size=(3, 3), stride=(1, 1), padding=same)
Sequential(
  (parallel_0): Conv2d(384, 384, kernel_size=(1, 3), stride=(1, 1), padding=same)
  (parallel_1): Conv2d(384, 384, kernel_size=(3, 1), stride=(1, 1), padding=same)
)
Conv2d(384, 384, kernel_size=(1, 3), stride=(1, 1), padding=same)
Conv2d(384, 384, kernel_size=(3, 1), stride=(1, 1), padding=same)


In [385]:
a=1
for i in list(inception_v3.modules[10]):
    if isinstance(i, nn.Sequential):
        for j in i :
            print(j)
            a+=1
            if a == 2 :
                break
            break

Conv2d(1280, 384, kernel_size=(3, 3), stride=(1, 1), padding=same)
Conv2d(1280, 448, kernel_size=(1, 1), stride=(1, 1), padding=same)
AvgPool2d(kernel_size=(1, 1), stride=(1, 1), padding=0)


In [241]:
for name in inception_v3.macro[10]:
    print(name)
#     break

parallel
{}
['conv', {'ksize': 1, 'chn': 320}]
['serial', {}, ['conv', {'ksize': [3, 3], 'chn': 384}], ['parallel', {}, ['conv', {'ksize': [1, 3], 'chn': 384}], ['conv', {'ksize': [3, 1], 'chn': 384}]]]
['serial', {}, ['conv', {'ksize': [1, 1], 'chn': 448}], ['conv', {'ksize': [3, 3], 'chn': 384}], ['parallel', {}, ['conv', {'ksize': [1, 3], 'chn': 384}], ['conv', {'ksize': [3, 1], 'chn': 384}]]]
['serial', {}, ['avg', {'ksize': 3, 'stride': 1}], ['conv', {'ksize': 1, 'chn': 192}]]


In [107]:
a=1
for i in inception_v3.modules[1].children():
    print(i,a)
    a+=1
#     break

Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), padding=same) 1
Sequential(
  (serial_0): Conv2d(192, 48, kernel_size=(1, 1), stride=(1, 1), padding=same)
  (serial_1): Conv2d(48, 64, kernel_size=(5, 5), stride=(1, 1), padding=same)
) 2
Sequential(
  (serial_0): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), padding=same)
  (serial_1): Conv2d(64, 96, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (serial_2): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=same)
) 3
Sequential(
  (serial_0): AvgPool2d(kernel_size=(1, 1), stride=(1, 1), padding=0)
  (serial_1): Conv2d(192, 32, kernel_size=(1, 1), stride=(1, 1), padding=same)
) 4


In [ ]:
class Parallel(nn.Module):
    def __init__(self, layer):
        super(Parallel, self).__init__()
        
        setattr(self, self.name, nn.Sequential(*layer))
        
        
    def forward(self,x):
        out = []
        for moduls in self.name.children():
            out.append(moduls(x))
#         out1 = self.layer1
            cat=torch.cat(out,dim=0)
        
        return cat

In [173]:
inception_v3.modules[10].

'Sequential'

In [171]:
for i in inception_v3.modules[10].children():
    print(i)

Conv2d(1280, 320, kernel_size=(1, 1), stride=(1, 1), padding=same)
Sequential(
  (serial_0): Conv2d(1280, 384, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (serial_1): Sequential(
    (parallel_0): Conv2d(384, 384, kernel_size=(1, 3), stride=(1, 1), padding=same)
    (parallel_1): Conv2d(384, 384, kernel_size=(3, 1), stride=(1, 1), padding=same)
  )
)
Sequential(
  (serial_0): Conv2d(1280, 448, kernel_size=(1, 1), stride=(1, 1), padding=same)
  (serial_1): Conv2d(448, 384, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (serial_2): Sequential(
    (parallel_0): Conv2d(384, 384, kernel_size=(1, 3), stride=(1, 1), padding=same)
    (parallel_1): Conv2d(384, 384, kernel_size=(3, 1), stride=(1, 1), padding=same)
  )
)
Sequential(
  (serial_0): AvgPool2d(kernel_size=(1, 1), stride=(1, 1), padding=0)
  (serial_1): Conv2d(1280, 192, kernel_size=(1, 1), stride=(1, 1), padding=same)
)


In [169]:
if isinstance(inception_v3.macro[10], list):
    for i in 
list(map(lambda x: print(x),inception_v3.macro[10]))

parallel
{}
['conv', {'ksize': 1, 'chn': 320}]
['serial', {}, ['conv', {'ksize': [3, 3], 'chn': 384}], ['parallel', {}, ['conv', {'ksize': [1, 3], 'chn': 384}], ['conv', {'ksize': [3, 1], 'chn': 384}]]]
['serial', {}, ['conv', {'ksize': [1, 1], 'chn': 448}], ['conv', {'ksize': [3, 3], 'chn': 384}], ['parallel', {}, ['conv', {'ksize': [1, 3], 'chn': 384}], ['conv', {'ksize': [3, 1], 'chn': 384}]]]
['serial', {}, ['avg', {'ksize': 3, 'stride': 1}], ['conv', {'ksize': 1, 'chn': 192}]]


[None, None, None, None, None, None]

In [188]:
{} in inception_v3.macro[10]

True

In [178]:
inception_v3.macro[10][4][0]

'serial'

In [158]:
for i in inception_v3.macro[10]:
    print(i)

parallel
{}
['conv', {'ksize': 1, 'chn': 320}]
['serial', {}, ['conv', {'ksize': [3, 3], 'chn': 384}], ['parallel', {}, ['conv', {'ksize': [1, 3], 'chn': 384}], ['conv', {'ksize': [3, 1], 'chn': 384}]]]
['serial', {}, ['conv', {'ksize': [1, 1], 'chn': 448}], ['conv', {'ksize': [3, 3], 'chn': 384}], ['parallel', {}, ['conv', {'ksize': [1, 3], 'chn': 384}], ['conv', {'ksize': [3, 1], 'chn': 384}]]]
['serial', {}, ['avg', {'ksize': 3, 'stride': 1}], ['conv', {'ksize': 1, 'chn': 192}]]


In [39]:
F_DATA_DIR = BASE_DIR+ 'code/util/dataset/flower_prepro.ipynb'

%run {F_DATA_DIR}

fd = FlowersDataset([96,96], [3,96,96])

Using device: cpu
Using PyTorch version: 1.9.0


In [40]:
CnnExtModel.set_macro('flower_preproc',
    ['serial',
        ['conv', {'ksize':3, 'stride':2, 'chn':6, 'actions':'#act'}]])

CnnExtModel.set_macro('flower_inception1',
    ['parallel',
        ['conv', {'ksize':1, 'chn':4, 'actions':'#act'}],
        ['conv', {'ksize':3, 'chn':6, 'actions':'#act'}],
        ['serial',
            ['conv', {'ksize':3, 'chn':6, 'actions':'#act'}],
            ['conv', {'ksize':3, 'chn':6, 'actions':'#act'}]],
        ['serial',
            ['avg', {'ksize':3, 'stride':1}],
            ['conv', {'ksize':1, 'chn':4, 'actions':'#act'}]]])

CnnExtModel.set_macro('flower_resize',
    ['parallel',
        ['conv', {'ksize':3, 'stride':2, 'chn':12, 'actions':'#act'}],
        ['serial',
            ['conv', {'ksize':3, 'chn':12, 'actions':'#act'}],
            ['conv', {'ksize':3, 'stride':2, 'chn':12, 'actions':'#act'}]],
        ['avg', {'ksize':3, 'stride':2}]])

CnnExtModel.set_macro('flower_inception2',
    ['parallel',
        ['conv', {'ksize':1, 'chn':8, 'actions':'#act'}],
        ['serial',
            ['conv', {'ksize':[3,3], 'chn':8, 'actions':'#act'}],
            ['parallel',
                ['conv', {'ksize':[1,3], 'chn':8, 'actions':'#act'}],
                ['conv', {'ksize':[3,1], 'chn':8, 'actions':'#act'}]]],
        ['serial',
            ['conv', {'ksize':[1,1], 'chn':8, 'actions':'#act'}],
            ['conv', {'ksize':[3,3], 'chn':8, 'actions':'#act'}],
            ['parallel',
                ['conv', {'ksize':[1,3], 'chn':8, 'actions':'#act'}],
                ['conv', {'ksize':[3,1], 'chn':8, 'actions':'#act'}]]],
        ['serial',
            ['avg', {'ksize':3, 'stride':1}],
            ['conv', {'ksize':1, 'chn':8, 'actions':'#act'}]]])

CnnExtModel.set_macro('flower_postproc',
    ['serial',
        ['avg', {'stride':6}],
        ['dropout', {'keep_prob':0.7}]])

In [41]:
CnnExtModel.set_macro('inception_flower',
    ['serial',
        ['custom', {'name':'flower_preproc', 'args':{'#act':'#act'}}],
        ['custom', {'name':'flower_inception1', 'args':{'#act':'#act'}}],
        ['custom', {'name':'flower_resize', 'args':{'#act':'#act'}}],
        ['custom', {'name':'flower_inception1', 'args':{'#act':'#act'}}],
        ['custom', {'name':'flower_resize', 'args':{'#act':'#act'}}],
        ['custom', {'name':'flower_inception2', 'args':{'#act':'#act'}}],
        ['custom', {'name':'flower_resize', 'args':{'#act':'#act'}}],
        ['custom', {'name':'flower_inception2', 'args':{'#act':'#act'}}],
        ['custom', {'name':'flower_postproc', 'args':{'#act':'#act'}}]])

In [42]:
conf_flower_LA = ['custom', {'name':'inception_flower', 'args':{'#act':'LA'}}]
model_flower_LA = CnnExtModel('model_flower_LA', fd, 
                         conf_flower_LA, dump_structure=True)

custom inception_flower
  serial
    custom flower_preproc
      serial
        1: conv, (3, 96, 96)=>[6, 48, 48]pm : 6x3x3x3+6 = 168
    custom flower_inception1
      parallel
        2: conv, [6, 48, 48]=>[4, 48, 48]pm : 4x6x1x1+4 = 28
        3: conv, [6, 48, 48]=>[6, 48, 48]pm : 6x6x3x3+6 = 330
        serial
          4: conv, [6, 48, 48]=>[6, 48, 48]pm : 6x6x3x3+6 = 330
          5: conv, [6, 48, 48]=>[6, 48, 48]pm : 6x6x3x3+6 = 330
        serial
          6: avg, [6, 48, 48]=>[6, 48, 48]
          7: conv, [6, 48, 48]=>[4, 48, 48]pm : 4x6x1x1+4 = 28
    custom flower_resize
      parallel
        8: conv, [20, 48, 48]=>[12, 24, 24]pm : 12x20x3x3+12 = 2172
        serial
          9: conv, [20, 48, 48]=>[12, 48, 48]pm : 12x20x3x3+12 = 2172
          10: conv, [12, 48, 48]=>[12, 24, 24]pm : 12x12x3x3+12 = 1308
        11: avg, [20, 48, 48]=>[20, 24, 24]
    custom flower_inception1
      parallel
        12: conv, [44, 24, 24]=>[4, 24, 24]pm : 4x44x1x1+4 = 180
        13: conv, 

In [43]:
model_flower_LA.exec_all(report=1)

Model model_flower_LA train started 


!!!!!!!!!!!
!! Layer !! .
!!!!!!!!!!! 

 Net(
  (layer1): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=48, out_features=5, bias=True)
  )
)

!!!!!!!!!!!!!!!
!! optimizer !! 
!!!!!!!!!!!!!!! 

 Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (10x27648 and 48x5)